<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/Anomaly_Transformer_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Anomaly Transformer:アソシエーション(連合,連想)の不一致を利用した時系列異常の検出**


---
- 時系列における**異常点**の**教師なし検出**は，モデルに**識別可能な基準**を導き出すことが要求される難しい問題である． 

- 従来の手法は，主にポイントワイズ(点別の)表現やペアワイズアソシエーション(二つ一組の比較)の学習によってこの問題に取り組んできたが，どちらも複雑なダイナミクス(挙動)を推論するには不十分であった．

- 近年、Transformerはポイントワイズ(点ごとの)表現とペアワイズアソシエーション(二つ一組の比較)の**統一的なモデリング(計算方法)**に大きな力を発揮しており、我々は**各時点の自己注視度分布**が**系列全体**との**豊かな関連性**を具現化できることを見出した。

- 我々の重要な観測は、**異常値の希少性**により、異常値から系列全体への非自明な関連付けを構築することが極めて困難であり、その結果、異常値の関連付けは主にその**隣接する時点に集中**することである。

- この**隣接集中の偏り**は、**正常点**と**異常点**を本質的に**区別できる**ような関連付けに基づく基準を意味し、我々はこれを**「関連性不一致」**という言葉で強調する。

- 技術的には、**連想不一致**を計算するための新しいメカニズムとして、㈶異常変換器 "を提案します。

- 関連性不一致の正規・非正規区別性を増幅させるために、**ミニマックス戦略**を考案しています。

- Anomaly Transformerは、サービス監視、宇宙・地球探査、水処理という3つのアプリケーションの教師なし時系列異常検出ベンチマーク6件において、最先端の結果を達成した。

---
- **論文**　Anomaly Transformer:アソシエーションの不一致を利用した時系列異常の検出
https://arxiv.org/abs/2110.02642

- ICLR2022の改訂を反映したモデル
https://github.com/spencerbraun/anomaly_transformer_pytorch/tree/main/archive

# **時系列異状検知にもTransformer**
### **3つの要点**

- いよいよ多変量時系列異状検知にもTransformerが現れました

- グラフも含めた深層学習により多変量の時系列の表現力は向上してきましたが、まだ単一時点に限ります

- Transformerのグローバルおよび長期の連関に対しての表現力を活かして、改造したAnomaly-Attentionを含む2分岐の構造で従来のSOTAを超える性能を確認しています

---
## **はじめに**
- いよいよ**多変量時系列異状検知**にも**Transformer**が現れました。

- 実際には、この論文の前に**GTA (Chen et al., 2021)vというグラフ構造により複数のIoTセンサーの関係を学習するモデル**にTransformerが時間軸のモデリングと異状検知の再構築基準に用いられていました。

- ブロック図を添付しておきます。他にも**TranAD**、**TiSAT**など続々と発表されています。

- 機会があれば、これらについても紹介したいと思います。

<img src="https://aisholar.s3.ap-northeast-1.amazonaws.com/post_images/4668dfff88e305d6790f787f3641c6da.png">

- 今回紹介するAnomaly Transformerは、**セルフアテンションメカニズムを異状検知用に改造したもの**です。

- 以前の記事の繰り返しになりますが、実世界のデータでは、異状検知を行いたい場合に、異常のデータはまれであり、**ラベリングを行うことは困難**であるため、ここでは**教師なし学習にフォーカス**します。

- この場合、教師データなしに、**正常／異常の判別基準**を作る必要があります。古典的な手法では、
 - **密度見積もり**
 - **クラスタリング手法**

がありますが、

- これらは**時間的要素を考慮しておらず**、見えていない実際のシナリオ(場面変化の順序)に一般化するのは難しいです。

- ニューラルネットワークによる表現学習の能力を利用して、最近の深層学習モデルは優れた成果を出しています。

- 主な手法カテゴリーでは、**回帰型ネットワーク**を通して**点的に表現学習**し、**再構成**あるいは**自己回帰タスク**により**自己教師学習**します。

- やはり、**異常データの少なさ**の為、**複雑な時間軸パターン**に対しては、**判別が難しく**なっています。

- また、**再構成**あるいは**予測誤差**は**時点毎**に計算されるので、**時間的なコンテキスト(前後の脈絡)の包括的な表現は難しく**なっています。

---
- 別の手法カテゴリーでは、**明示的な連関(association)モデリング**により異状を検知します。

 - **ベクトル自己回帰(VAR)** VARは、時系列データに対して、**変数間の相関**や**影響**を**推定する**ことができます。
 - **状態空間モデル**は**系の振る舞いを表す数学的モデルの一種**
 - **グラフニューラル ネットワーク(GNN)**グラフとして表現できるデータを処理するための人工ニューラル ネットワーク
 がこのカテゴリーに属します。

- 以前紹介したように、グラフとして表現できるデータを処理するための人工ニューラル ネットワークは多変量時系列の動的なグラフの学習に適用されています。

- 表現能力は向上していますが、まだ**単一時点のモデル**に限定されています。

- 一方**部分シーケンスベースの手法**は、**部分シーケンス間の類似性**を計算することにより**異常＊＊*を検知します。

- しかし、これらの手法(部分シーケンスベースの手法)では、**各時点と系列全体の詳細な時間的連関を捕捉すること**ができません。

---

- この論文の手法では、Transformerを時系列異状検知の教師なし学習に適用しています。

- Transformerがひろく適用されているのは、**グローバルな表現**と、**長期の連関を統一的に表現できる**ためです。

- 時系列に適用した場合に、セルフアテンションマップは**各時点の時間的連関を表現**します。

- これを**系列連関(series-association)**と名付けます。

- さらに、**異常の希少性**と**正常なパターンの優位性**のために、異常が**系列全体との強い関連を構築すること**はより困難であることが観察されます。

- 異常の関連は、**連続性**のために**同様の異常パターン**を含む可能性が高い隣接する時点に集中する必要があります。

- このような隣接濃度の誘導バイアスは、**事前連関(prior-association)**と呼ばれます。

- 対照的に、支配的な正常の時点は、**隣接する領域**に限定されるのではなく、**系列全体との有益な関連**を発見することができます。

- この観察に基づいて、**関連分布の固有の正常-異常識別可能性**を利用しようとします。

- これにより、**各時点の新しい異常基準**が導き出されます。

- これは、**各時点の事前連関**とその**系列連関の間の距離**によって**定量化**され、**連関不一致(Association Discrepancy)**と呼ばれます。

- 前述のように、**異常の関連は隣接集中**している可能性が高いため、**異常は正常の時点よりも連関不一致が小さくなります。**

---
- 教師なし時系列異常検出にTransformerを導入し、連関学習用のAnomalyTransformerを提案しています。

- **連関不一致(Association Discrepancy)**を計算するために、セルフアテンションメカニズムをAnomaly-Attentionに刷新します。

- これには、**各時点の事前連関**と**系列連関**を**それぞれモデル化**する**2つの分岐構造**が含まれています。

- **事前連関**では、学習可能なガウスカーネルを使用して、**各時点の隣接する濃度の誘導バイアスを提示**します。

- 一方、**系列連関**は、**生の系列**から学習した**セルフアテンションの重み**に対応します。

- さらに、**ミニマックス戦略**が**2つの分岐間**に適用されます。

- これにより、**連関不一致**の**正常と異常の識別可能性が増幅**され、**新しい連関ベースの基準**がさらに導き出されます。 


---

この論文の貢献は、次の3つです。

- **連関の不一致**の主要な観察に基づいて、Anomaly-Attentionメカニズムを備えたAnomaly-Transformerを提案します。

 - これにより、**事前連関**と**系列連関**を**同時にモデル化**して、**連関の不一致を具体化**できます。

- **連関不一致**の**正常**と**異常**の**識別可能性を拡大**し、さらに**新しい連関ベースの検出基準を導出**するための**ミニマックス戦略**を提案します。

- **Anomaly Transformer**は、3つの実際のアプリケーションの**6つのベンチマークでSOTAの異状検出結果**を実現します。 
 - 広範な切り分けと洞察に満ちたケーススタディを行います。

---
# **まとめ**
この論文は、**教師なし時系列異常検出問題**を研究しています。 

- 以前の方法とは異なり、Transformersによる**より有益な時間軸の連関を学習**します。 

- 重要である**連関不一致の観察**に基づいて、Anomaly Transformerを提案しています。

- これには、**連関不一致を具体化**するための**2分岐構造**の**Anomaly-Attention**が含まれます。 

- **ミニマックス戦略**は、**正常な時点と異常な時点の違い**を**さらに増幅する**ために採用されています。

**連関不一致**を導入することにより、**再構成の性能**と**連関不一致を連携させる連関ベースの基準**を提案します。 

- Anomaly Transformerは、経験的研究のデータセットで広く評価し、**SOTAの結果を確認**しています。

- 将来的には、**自己回帰**や**状態空間モデル**の古典的な分析に照らして、Anomaly Transformerの理論的な研究を行うとしています。

---

- 時系列異状検知にもTransformer

https://ai-scholar.tech/articles/error-detection/Anomaly_Transforme

# **ベクトル自己回帰（VectorAutoregression, VAR）**

---
- 時系列データを分析するための方法の一つです。VARは、時系列データに対して、**変数間の相関**や**影響**を**推定する**ことができます。

- VARモデルは、**複数の変数**に対して、時系列データを表す式を導出することで、それらの変数の間の関係を表すことができます。

---

- 具体的には、VARモデルは、以下の式で表されます。

 - $$ Y_t = c + A_1Y_{t-1} + A_2Y_{t-2} + \dots + A_pY_{t-p} + \varepsilon_t $$


 - $Y_t$は**時刻$t$における変数の値**
 - **$A_1, A_2, \dots, A_p$は係数を表すパラメータ**
 - **$c$は定数**
 - **$\varepsilon_t$は誤差項**
を表します。

- VARモデルでは、**各変数が過去の自身の値**や**他の変数の値**によって**影響を受ける**ことが考えられます。

- VARモデルは、**複数の変数を同時に扱うことができる**ため、**変数間の相関**や**影響を推定する**ことができます。

- また、VARモデルは、**推定された係数**を用いて、**将来の値を予測すること**もできます。

---
- **ベクトル自己回帰（VAR）**とは、**複数の時系列変数間**の**線形相互依存関係**を記述するために用いられる統計モデルである。

- **単一の時系列**のみを考慮する**一変量自己回帰モデル（AR）**を**多変量に拡張したもの**である。

- VARモデルでは、各変数は**自身のラグ値**と**モデル内の他の変数のラグ値**の**一次関数**となる。

- つまり、**各変数の現在値**は、**その過去の値**と、**モデル内の他の変数の過去の値に依存**する。

---
- 以下は、$X$と$Y$の2つの変数を使ったVARモデルの例である。

- $X(t) = a1 * X(t-1) + a2 * Y(t-1) + error(t) Y(t) = b1 * X(t-1) + b2 * Y(t-1) + error(t)$ この例では、$X4 と $Y$ の 2 つの変数からなる $VAR$ モデルの例である。

- この例では、$X$の**現在値**$（X（t））$は、$X$の**前回値**$（X（t-1））$と$Y$の**前回値**$（Y（t-1））$の関数であり、$Y$の**現在値**$（Y（t））$は、$X$の**前回値**$（X（t-1））$と$Y$の**前回値**$（Y（t-1））$の関数である。

- **誤差**$（error(t)）$は、**モデルに含まれない他の要因の影響**を表す。

- $VAR$モデルは、複数の時系列間の**相互依存関係を分析**し、変数の**将来の値について予測**を行うために使用することができます。

- 経済や金融などの分野では、**GDP**や**インフレ**、**雇用**などの**経済指標間の関係**を調べるためによく利用されている。

- 例えば、ある企業の株価$（X）$と原油価格$（Y）$の関係を調べるのに、$VAR$モデルが使われるかもしれません。

- このモデルは、**原油価格の変動**が**株価の変動**に**どの程度影響を与えるか**、あるいは逆にどの程度影響を与えるかを明らかにし、**原油価格の予想変動**に基づいて**将来の株価を予測する**ために使用することができる。

# **状態空間モデル**

---
- 状態空間モデルとは、時系列データを扱うための数理モデルの一種です。

- 状態空間モデルでは、時刻 $t$ の状態を表す状態変数 $x_t$、時刻 $t$ の観測値を表す観測変数 $y_t$、および時刻 $t$ の状態から時刻 $t+14$ の状態への移行を表す移行関数 $f_t4$ と、時刻 $t4$ の状態から時刻 $t$ の観測値への変換を表す観測関数 $h_t$ を用いて、次のような方程式で表されます。

- $x_t = f_t(x_{t-1}, u_t)$

- $y_t = h_t(x_t, v_t)$

- ここで、$u_t$ は時刻 $t$ の状態に対する外部入力を表し、$v_t$ は時刻 $t$ の観測値に対する雑音を表します。

- 状態空間モデルを使用することで、時系列データを効率的に解析することができます。

- また、状態空間モデルを用いて、将来のデータを予測することもできます。

---

- 状態空間モデルとは、**系の振る舞いを表す数学的モデルの一種**です。

- 状態空間モデルは、**状態**、**状態遷移**、**観測**を表す式で構成されます。

- **状態**は、系が持つ情報を表します。
- 例えば、自動車の**状態**は、**スピード、位置、角度など**で表すことができます。
- **状態**は、通常は多次元で表されます。

- **状態遷移**は、**系の状態が時間の経過とともにどのように変化するか**を表します。
- 例えば、自動車の状態遷移は、**スピードによって位置がどのように変化するか**、**角度によってスピードがどのように変化するか**などで表すことができます。

- **観測**は、**系から得られる情報**を表します。
- 例えば、自動車から得られる**観測**は、**車体の位置やスピードなど**です。

- **状態空間モデル**は、**系の状態**や**状態遷移**を**数学的に表す**ことで、**系の振る舞い**を**予測すること**ができます。

- 状態空間モデルは、さまざまな分野で使われており、**自動車の自己走行**や、**工場の生産ライン**などで活用されています。

# **Graph neural network(GNN,グラフ ニューラル ネットワーク)**

---
wiki
- グラフニューラルネットワークについて教えて

---
- グラフ ニューラル ネットワークは、**グラフとして表現できるデータ**を処理するための人工ニューラル ネットワークのクラスです。 

- **「幾何学的深層学習」**のより一般的な主題では、特定の既存のニューラル ネットワーク アーキテクチャは、適切に定義されたグラフで動作する GNN として解釈できます。

---
ChatGPT
- グラフニューラルネットワークについて教えて

---

- グラフニューラルネットワーク (Graph Neural Network, GNN) は、**データを表すグラフ構造上で処理を行うニューラルネットワークのモデル**です。
- **GNN**は、**ノード**や**エッジ**に**属性**や**特徴量**を**割り当て**、それらをmもとに**グラフ全体を表現すること**ができます。

- GNNは、**グラフ構造を持つデータ**を扱うタスクに特に有効です。

 -たとえば、**社会ネットワーク**や**自然言語処理タスク**では、**人**や**単語**などの**データ**を**ノード**として、それらを結ぶ**リンク**や**関係**を**エッジ**として**表現する**ことができます。
 
- **GNN**はこのような**グラフ構造**を利用して、**グラフ全体を表す表現を学習**することができます。

- GNNは、様々なタスクに応用されています。
 - たとえば、**グラフ上でのクラス分類タスク**や**回帰タスク**、**セマンティックセグメンテーションタスク**などが挙げられます。

- また、GNNは**自然言語処理タスク**でも用いられることがあります。

- GNNには、**畳み込みニューラルネットワーク (Convolutional Neural Network, CNN) を拡張したモデル**や、**リカレントニューラルネットワーク (Recurrent Neural Network, RNN)** を拡張したモデルなどがあります。

- これらのモデルは、**グラフ上でのデータ処理を行う**ために、**特別な畳み込み**や**再帰演算を導入**しています。

- GNNは、近年注目されており、様々なタスクで有効な結果が得られています。その一方で、**GNNは計算が複雑である**ことや、**データの欠損**や**ノイズの影響***を**受けやすい**という**欠点**もあります。

---
- グラフニューラルネットワークについてわかりやすく

---

- グラフニューラルネットワーク（Graph Neural Network、GNN）は、**グラフ構造を持つデータ**を扱うためのニューラルネットワークです。

- グラフは、データを**ノード（点）**と**エッジ（辺）**で表すことができます。

- GNNは、**グラフ上**の**ノード同士**をつなぐ**エッジを考慮**して、**各ノードの特徴量を抽出**します。

- GNNは、様々なタスクで活用されています。

- 例えば、**自然言語処理タスク**では、**文章**を**単語**の**グラフ構造**として表現し、GNNを使って**文章の意味を理解する**ことができます。

 - また、**化学データ**や**社会ネットワークデータ**などを扱う場合にも、GNNを使ってそのデータの構造を理解することができます。

- GNNのモデル構造は、普通のニューラルネットワークと大きく異なります。

- 通常のニューラルネットワークでは、**入力データ**を**一つの行列**に**整形**して入力しますが、GNNでは、**各ノード**の情報を**行列として入力**し、その**行列同士**をつなぐ**エッジの情報**も**入力**します。

- その後、**GNN**では、**各ノードの情報**を**更新**するために、その**ノード**とつながっている**エッジの情報**を**考慮した計算**を行います。

- GNNは、**複雑なグラフ構造を扱うタスクでは有効**ですが、**その計算は複雑**であるため、**計算量が大きくなる**ことがあります。

- また、GNNでは、**グラフ構造**を**正確に表現することが重要**であるため、**グラフの構造**を**正しく理解**することが求められます。

---
- グラフニューラルネットワークについて例を用いて説明して

---

- グラフニューラルネットワーク (GNN) は、グラフ構造を扱うためのニューラルネットワークです。

- GNN は、ノードとエッジを含むグラフ構造を入力とし、各ノードの情報を出力します。

- 例えば、ある企業内における**社員のグラフ**を考えます。

 - このグラフでは、**各社員がノード**、**社員間の上司と部下の関係がエッジ**で表されています。
 
 - GNN を用いることで、**各社員のタイトルや役職などの情報**を出力することができます。

 - GNN の計算は、**各ノードから出るエッジ**を辿って、**隣接するノードの情報**を取得し、**各ノードの情報を更新**することで行われます。

- このように、GNN は**グラフ構造上でのデータ伝播を表現**することができます。

- GNN の一般的なアプリケーションとしては、**様々なグラフ構造**を持つデータセットを**分類**や**予測するタスク**、**グラフ上での推論**や**探索タスク**、グラフ上での**ノード分類**や**エッジ生成タスク**などがあります。

---
- グラフニューラルネットワークについて他の例を用いて説明して

- グラフニューラルネットワーク (Graph Neural Network, GNN) は、グラフ構造を持つデータを扱うためのニューラルネットワークモデルです。

- 典型的な使用例としては、以下のようなものがあります。

- **化学反応における分子の構造**や**反応性を予測する**ためのモデル構築

- **ソーシャルネットワークや航空ルート**のような、**ノードとエッジで構成されるネットワークを分析する**ためのモデル構築

- 自然言語処理タスクで、**文章や文書を構成する単語間の関係**を表すグラフを扱うためのモデル構築

- **医学画像解析で、解剖学的構造を表すグラフ**を扱うためのモデル構築

- GNN は、通常のニューラルネットワークと同様に、**入力データから出力を予測するモデル**ですが、**入力データ**が**グラフ構造を持つこと**が特徴です。

- GNN は、**グラフ上のノードやエッジに対応する特徴量**を学習し、それらの**特徴量**を元に**グラフ全体の特徴を抽出**することができます。

- GNN の実装には、複数の種類のモデルがあります。

- その一つが、**畳み込みニューラルネットワーク (Convolutional Neural Network, CNN)** を拡張したものです。

- このモデルでは、グラフ上のノードやエッジに対応する**特徴量を畳み込み層**を通じて学習します。

- 他にも、**リカレントニューラルネットワーク (Recurrent Neural Network, RNN)** を拡張したものや、**マルチヘッド注意機構 (Multi-Head Attention Mechanism)** を用いたものなどがあります。

# **ミニマックス戦略**

---
ChatGPT
- ミニマックス戦略について例を出してわかりやすく教えて
---
- ミニマックス戦略とは、ゲーム理論において、**プレイヤーが自分ができる中で最も悪い結果を最小化する戦略をとること**を指します。

- この戦略は、**相手がどのような戦略**をとっても、**自分ができる限り最も悪い結果**を**最小化**することで、**最も良い結果を得ることができる**と考えられています。

- 例えば、二人でカードゲームをするとします。

---
- このゲームでは、**二人とも 1 枚のカードを引き、カードの大小で勝負をする**ものとします。

- このとき、**プレイヤー A** は、**自分ができる限り最も悪い結果を最小化する**ために、**相手が引くカードの大小を考慮**して、**自分が引くカードを選ぶ**ことにします。

- つまり、**相手が引くカードが大きい場合は、小さいカードを引く**ことで、**自分ができる限り最も悪い結果を最小化**し、**最も良い結果を得ることができます。**

- これが、**ミニマックス戦略**の例です。ゲーム理論においては、このような戦略をとることで、**最も良い結果を得ることができる**と考えられています。

---
ChatGPT
- Tell me about the minimax strategy in a simple way with examples.
- （訳）ミニマックス戦略について例を出してわかりやすく教えて
---
- ミニマックス戦略とは、チェス、三目並べ、囲碁などの**二人用ゲーム**で用いられる**意思決定アルゴリズムの一つ**である。

- プレイヤーにとって**最大の損失を最小限に抑えるという考え**に基づいており、**"ミニマックス原理 "**とも呼ばれる。

- その仕組みはこうだ。

---
- **2人用のゲーム**では、各プレイヤーは**各ターンで可能な限り最善の手を打つ**ことで、**勝つチャンスを最大化**しようとする。

- 同時に、各プレイヤーは、**相手の可能な動きを予測**し、**それに対抗する**ことによって、**相手の勝利のチャンスを最小**にしようとする。

- そのために、ミニマックスアルゴリズムは、**すべての可能な手**と**その結果**を、**手を打つプレーヤー**と**相手**の**両方について検討**する。

- そして、**プレーヤーにとって損失が最小**になり、**相手にとっても損失が最大**になるような手を選択する。

- 三目並べのような簡単なゲームでミニマックス戦略がどのように機能するか、例を挙げてみましょう。

- プレイヤー1（X）が最初の手を打ち、ボードの左上隅に自分のXを置きます。

- プレイヤー2（O）は、自分のOをどこに置くかを決めなければなりません。ミニマックス戦略を使って、可能なすべての手とその結果を検討します。

- 例えば、プレイヤー2が○をボードの中央に置いた場合、プレイヤー1は○を右上と左下に置けば勝てる可能性があります。

- **プレイヤー2はその損失を最小限**にするため、右上に○を置き、**プレイヤー1の勝利の可能性を阻み**、**自分の勝利の可能性を設定する**ことができるのです。

- ミニマックスアルゴリズムは、**すべての可能な手**と**その結果を考慮**することで、**プレイヤーが勝つ可能性を最大**にし、**負ける可能性を最小**にする**戦略的決定を行うこと**を支援します。

In [ ]:
!pip install hydra-core
!pip install omegaconf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 117 kB 55.6 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=d6dc0a0a336e9685cda32751647d959df0b2d56e02fe0e475c6895490e9748ef
  Stored in directory: /root/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c47048baf9a1617c485
Successfully built antlr4-python3-runtime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import hydra#Hydraというライブラリを用いてパラメータを管理

#OmegaConf は、Python の Configuration フレームワークのひとつ
#上記のHydra が低レイヤー API に利用している
from omegaconf import DictConfig
from omegaconf.omegaconf import OmegaConf

import numpy as np
from sklearn.datasets import load_digits

import matplotlib.pyplot as plt



In [ ]:
time_series = np.random.rand(1000, 300)
print("1行目のtime_series形状=",time_series.shape,
      " \n 1行目のtime_series=",time_series)
print("*"*100)
time_series[500:560, 100:200] += 0.3
print("2行目のtime_series形状=",time_series.shape,
      " \n 2行目のtime_series=",time_series)
print("*"*100)
time_series = torch.from_numpy(time_series)
print("3行目のtime_series形状=",time_series.shape,
      " \n 3行目のtime_series=",time_series)
print("*"*100)

1行目のtime_series形状= (1000, 300)  
 1行目のtime_series= [[0.99578645 0.50063042 0.03840311 ... 0.0335864  0.12535074 0.23522376]
 [0.49947668 0.83666722 0.04225995 ... 0.14145073 0.41259859 0.43452688]
 [0.48863532 0.27779981 0.34093389 ... 0.84979627 0.91847703 0.75824464]
 ...
 [0.91191668 0.17719829 0.38155875 ... 0.87007334 0.83177898 0.03549149]
 [0.74618056 0.75618591 0.49562811 ... 0.88723528 0.46931805 0.85554166]
 [0.9495657  0.71692282 0.69257092 ... 0.05559671 0.57177414 0.28263561]]
****************************************************************************************************
2行目のtime_series形状= (1000, 300)  
 2行目のtime_series= [[0.99578645 0.50063042 0.03840311 ... 0.0335864  0.12535074 0.23522376]
 [0.49947668 0.83666722 0.04225995 ... 0.14145073 0.41259859 0.43452688]
 [0.48863532 0.27779981 0.34093389 ... 0.84979627 0.91847703 0.75824464]
 ...
 [0.91191668 0.17719829 0.38155875 ... 0.87007334 0.83177898 0.03549149]
 [0.74618056 0.75618591 0.49562811 ... 0.88723528 0.4693

In [ ]:
p = torch.from_numpy(np.abs(np.indices((100,100))[0] - np.indices((100,100))[1]))
print("p_shape=",p.shape,"\n p=",p)
print("*"*100)
sigma = torch.ones(100).view(100, 1) * 2
print("sigma_shape=",sigma.shape,"\n sigma=",sigma)


p_shape= torch.Size([100, 100]) 
 p= tensor([[ 0,  1,  2,  ..., 97, 98, 99],
        [ 1,  0,  1,  ..., 96, 97, 98],
        [ 2,  1,  0,  ..., 95, 96, 97],
        ...,
        [97, 96, 95,  ...,  0,  1,  2],
        [98, 97, 96,  ...,  1,  0,  1],
        [99, 98, 97,  ...,  2,  1,  0]])
****************************************************************************************************
sigma_shape= torch.Size([100, 1]) 
 sigma= tensor([[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
     

In [ ]:
P = torch.ones(10,10) * torch.arange(10).view(10,1)
print("P.shape=",P.shape,"\nP=",P)

P.shape= torch.Size([10, 10]) 
P= tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
        [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
        [6., 6., 6., 6., 6., 6., 6., 6., 6., 6.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
        [8., 8., 8., 8., 8., 8., 8., 8., 8., 8.],
        [9., 9., 9., 9., 9., 9., 9., 9., 9., 9.]])


In [ ]:
S = torch.ones(10,10) * torch.arange(10).view(1,10)
print("S.shape=",S.shape,"\nS=",S)

S.shape= torch.Size([10, 10]) 
S= tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])


In [ ]:
lambda row: F.kl_div(P[row,:], S[row,:]) + F.kl_div(S[row,:], P[row,:])
print("P.shape=",P.shape,"\nP=",P)
print("*"*100)
print("S.shape=",S.shape,"\nS=",S)
print("*"*100)
print("\nF=",F)

P.shape= torch.Size([10, 10]) 
P= tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
        [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
        [6., 6., 6., 6., 6., 6., 6., 6., 6., 6.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
        [8., 8., 8., 8., 8., 8., 8., 8., 8., 8.],
        [9., 9., 9., 9., 9., 9., 9., 9., 9., 9.]])
****************************************************************************************************
S.shape= torch.Size([10, 10]) 
S= tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        [0., 1., 2., 3., 4., 5

In [ ]:
torch.arange(0, 10, 0.1)

tensor([0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
        0.9000, 1.0000, 1.1000, 1.2000, 1.3000, 1.4000, 1.5000, 1.6000, 1.7000,
        1.8000, 1.9000, 2.0000, 2.1000, 2.2000, 2.3000, 2.4000, 2.5000, 2.6000,
        2.7000, 2.8000, 2.9000, 3.0000, 3.1000, 3.2000, 3.3000, 3.4000, 3.5000,
        3.6000, 3.7000, 3.8000, 3.9000, 4.0000, 4.1000, 4.2000, 4.3000, 4.4000,
        4.5000, 4.6000, 4.7000, 4.8000, 4.9000, 5.0000, 5.1000, 5.2000, 5.3000,
        5.4000, 5.5000, 5.6000, 5.7000, 5.8000, 5.9000, 6.0000, 6.1000, 6.2000,
        6.3000, 6.4000, 6.5000, 6.6000, 6.7000, 6.8000, 6.9000, 7.0000, 7.1000,
        7.2000, 7.3000, 7.4000, 7.5000, 7.6000, 7.7000, 7.8000, 7.9000, 8.0000,
        8.1000, 8.2000, 8.3000, 8.4000, 8.5000, 8.6000, 8.7000, 8.8000, 8.9000,
        9.0000, 9.1000, 9.2000, 9.3000, 9.4000, 9.5000, 9.6000, 9.7000, 9.8000,
        9.9000])

In [ ]:
print(torch.exp(p.pow(2) / (sigma)))
print(torch.exp(p.pow(2) / (sigma)).shape)

tensor([[1.0000, 1.6487, 7.3891,  ...,    inf,    inf,    inf],
        [1.6487, 1.0000, 1.6487,  ...,    inf,    inf,    inf],
        [7.3891, 1.6487, 1.0000,  ...,    inf,    inf,    inf],
        ...,
        [   inf,    inf,    inf,  ..., 1.0000, 1.6487, 7.3891],
        [   inf,    inf,    inf,  ..., 1.6487, 1.0000, 1.6487],
        [   inf,    inf,    inf,  ..., 7.3891, 1.6487, 1.0000]])
torch.Size([100, 100])


In [ ]:
gaussian = torch.normal(p.float(), sigma)
print("1行目gaussian.shape=",gaussian.shape,"\ngaussian=",gaussian)
print("*"*100)
gaussian /= gaussian.sum(dim=-1).view(-1, 1)
print("2行目gaussian.shape=",gaussian.shape,"\ngaussian=",gaussian)

1行目gaussian.shape= torch.Size([100, 100]) 
gaussian= tensor([[ 0.9198,  3.7110,  3.4218,  ..., 97.1249, 98.4036, 99.9262],
        [-0.9890, -1.0475,  0.5405,  ..., 99.9480, 95.3661, 97.4227],
        [ 0.2086,  4.1782,  0.7257,  ..., 91.9734, 93.8262, 98.1946],
        ...,
        [96.7209, 98.8593, 93.2461,  ..., -1.9867, -0.7331,  1.9060],
        [97.9027, 99.5482, 93.3440,  ..., -1.8998,  0.3557,  1.3289],
        [99.3308, 98.8553, 98.2764,  ..., -0.6870,  0.9395, -2.0383]])
****************************************************************************************************
2行目gaussian.shape= torch.Size([100, 100]) 
gaussian= tensor([[ 1.8459e-04,  7.4475e-04,  6.8671e-04,  ...,  1.9492e-02,
          1.9748e-02,  2.0054e-02],
        [-2.0326e-04, -2.1529e-04,  1.1108e-04,  ...,  2.0541e-02,
          1.9599e-02,  2.0022e-02],
        [ 4.3599e-05,  8.7330e-04,  1.5169e-04,  ...,  1.9224e-02,
          1.9611e-02,  2.0524e-02],
        ...,
        [ 2.0468e-02,  2.0921e-02,  1

In [ ]:
gaussian[0,:].sum()

tensor(1.)

In [ ]:
print("形状=",len(torch.ones(5,10)/ torch.ones(5,10).sum(dim=-1).view(-1,1)))
print("\n値=",torch.ones(5,10)/ torch.ones(5,10).sum(dim=-1).view(-1,1))

形状= 5

値= tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000]])


In [ ]:
torch.ones(5,10).sum(dim=-1)

tensor([10., 10., 10., 10., 10.])

In [ ]:
torch.normal(torch.arange(0,100).float())

tensor([-1.1174,  1.7723,  2.4031,  2.7874,  2.8778,  3.3509,  7.0244,  8.0379,
         7.4041,  8.1695,  9.2901, 12.8908, 12.8003, 12.5679, 14.0763, 15.6744,
        15.2470, 15.9080, 19.4755, 19.4249, 19.6383, 21.0976, 22.6707, 23.3365,
        24.4910, 24.5622, 27.6387, 26.3944, 27.4720, 30.7173, 32.0237, 32.4980,
        32.8682, 32.0688, 33.3107, 34.6450, 35.6070, 37.4376, 36.8804, 38.1887,
        40.8927, 40.6629, 43.5812, 42.3893, 41.8557, 43.5534, 44.8637, 47.8431,
        47.7764, 49.1840, 51.1752, 52.0577, 51.1101, 52.4869, 54.2716, 54.6024,
        55.6377, 57.9288, 57.9156, 60.4437, 58.9162, 60.5936, 60.4806, 61.9633,
        63.6134, 65.3084, 65.3482, 66.6408, 69.5445, 70.0024, 70.8892, 69.8721,
        71.3693, 73.0641, 74.4419, 75.8983, 76.4714, 77.2696, 79.4129, 78.4817,
        79.5877, 80.8206, 82.3307, 82.5360, 83.8835, 83.9322, 84.8979, 88.2499,
        86.8648, 89.2311, 89.1574, 91.5336, 91.5168, 89.4169, 95.2944, 94.9668,
        95.1676, 96.7156, 98.9482, 99.80

In [ ]:
class AnomalyAttention(nn.Module):
    def __init__(self, seq_dim, in_channels, out_channels):
        super(AnomalyAttention, self).__init__()
        self.W = nn.Linear(in_channels, out_channels, bias=False)
        self.Q = self.K = self.V = self.sigma = torch.zeros((seq_dim, out_channels))
        self.d_model = out_channels
        self.n  = seq_dim
        self.P = torch.zeros((seq_dim, seq_dim))
        self.S = torch.zeros((seq_dim, seq_dim))

    def forward(self, x):

        self.initialize(x) # does this make sense?
        self.P = self.prior_association()
        self.S = self.series_association()
        Z = self.reconstruction()

        return Z

    def initialize(self, x):
        # self.d_model = x.shape[-1]
        self.Q = self.K = self.V = self.sigma = self.W(x)
        

    def prior_association(self):
        p = torch.from_numpy(
            np.abs(
                np.indices((self.n,self.n))[0] - 
                np.indices((self.n,self.n))[1]
                )
            )
        gaussian = torch.normal(p.float(), self.sigma[:,0].abs())
        gaussian /= gaussian.sum(dim=-1).view(-1, 1)

        return gaussian

    def series_association(self):
        return F.softmax((self.Q @ self.K.T) / math.sqrt(self.d_model), dim=0)

    def reconstruction(self):
        return self.S @ self.V

    def association_discrepancy(self):
        return F.kl_div(self.P, self.S) + F.kl_div(self.S, self.P) #not going to be correct dimensions



In [ ]:
class AnomalyTransformerBlock(nn.Module):
    def __init__(self, seq_dim, feat_dim):
        super().__init__()
        self.seq_dim, self.feat_dim = seq_dim, feat_dim
       
        self.attention = AnomalyAttention(self.seq_dim, self.feat_dim, self.feat_dim)
        self.ln1 = nn.LayerNorm(self.feat_dim)
        self.ff = nn.Sequential(
            nn.Linear(self.feat_dim, self.feat_dim),
            nn.ReLU()
        )
        self.ln2 = nn.LayerNorm(self.feat_dim)
        self.association_discrepancy = None

    def forward(self, x):
        x_identity = x 
        x = self.attention(x)
        z = self.ln1(x + x_identity)
        
        z_identity = z
        z = self.ff(z)
        z = self.ln2(z + z_identity)

        self.association_discrepancy = self.attention.association_discrepancy().detach()
        
        return z

In [ ]:
class AnomalyTransformer(nn.Module):
    def __init__(self, seqs, in_channels, layers, lambda_):
        super().__init__()
        self.blocks = nn.ModuleList([
            AnomalyTransformerBlock(seqs, in_channels) for _ in range(layers)
        ])
        self.output = None
        self.lambda_ = lambda_
        self.assoc_discrepancy = torch.zeros((seqs, len(self.blocks)))
    
    def forward(self, x):
        for idx, block in enumerate(self.blocks):
            x = block(x)
            self.assoc_discrepancy[:, idx] = block.association_discrepancy
        
        self.assoc_discrepancy = self.assoc_discrepancy.sum(dim=1) #N x 1
        self.output = x
        return x

    def loss(self, x):
        l2_norm = torch.linalg.matrix_norm(self.output - x, ord=2)
        return l2_norm + (self.lambda_ * self.assoc_discrepancy.mean())

    def anomaly_score(self, x):
        score = F.softmax(-self.assoc_discrepancy, dim=0)

In [ ]:
model = AnomalyTransformer(seqs=1000, in_channels=300, layers=3, lambda_=0.1)

In [ ]:
model(time_series.float())

NameError: ignored

In [ ]:
model.loss(time_series)

TypeError: ignored

In [ ]:

import torch 
import torch.nn as nn 
import torch.nn.functional as F  

# コンフィギュレーション管理のための hydra ライブラリをインポートする
import hydra 

# omegaconf ライブラリから DictConfig と OmegaConf をインポート
# コンフィギュレーション管理用
from omegaconf import DictConfig
from omegaconf.omegaconf import OmegaConf

# numpy と load_digits 関数をインポート from sklearn.datasets
import numpy as np
#sklearn.datasetsからload_digitsをインポート
from sklearn.datasets import load_digits


# プロットするために matplotlib ライブラリから pyplot をインポート
import matplotlib.pyplot as plt 

# 形状が (1000, 300) のランダムな時系列を作成する。
time_series = np.random.rand(1000, 300)

# 時系列の特定の部分集合に 0.3 という値を追加する
time_series[500:560, 100:200] += 0.3

# 時系列を PyTorch のテンソルに変換する
time_series = torch.from_numpy(time_series)

# (100,100)の配列のインデックスからテンソルを作成する。
# そしてインデックス間の差の絶対値を計算する
p = torch.from_numpy(np.abs(np.indices((100,100))[0] - np.indices((100,100))[1]))

# サイズ (100,) が 1 で満たされたテンソルを作成します。
# そして、それを (100, 1) の形状になるように整形する。
sigma = torch.ones(100).view(100, 1) * 2

# 0-9の値で満たされた(10,10)テンソルを作成する。
# 形状が (10,1) になるように整形する。
P = torch.ones(10,10) * torch.arange(10).view(10,1)

# 0-9の値で満たされた(10,10)テンソルを作成する。
# 形状が (1,10) になるように再形成される
S = torch.ones(10,10) * torch.arange(10).view(1,10)。

#P と P の行の間の KL ダイバージェンスを計算する # ラムダ関数を定義する。
#P の行と S の行の間の KL 発散と、S の行と S の行の間の KL 発散を計算する # ラムダ関数を定義する。
#S の行と P の行の間の KL 発散を計算する # ラムダ関数を定義する。
lambda row:F.kl_div(P[row,:], S[row,:]) + F.kl_div(S[row,:], P[row,:])

# Pテンソルを表示する
P

# 0から10までの値を0.1間隔で並べたテンソルを作成する。
torch.arange(0, 10, 0.1)

# (p^2 / σ)の指数を計算する。
torch.exp(p.pow(2) / (sigma))

# 平均 p、標準偏差シグマでガウス分布を作成する。
gaussian = torch.normal(p.float(), sigma)

# ガウス分布の和が1になるように正規化する
gaussian /= gaussian.sum(dim=-1).view(-1, 1)

# ガウス分布の最初の行の和を表示する
gaussian[0,:].sum()

# サイズ (5,10) が 1 で埋め尽くされたテンソルを作成する。
# そして、和が1になるように正規化する
# 各行の合計を表示
torch.ones(5,10).sum(dim=-1)。

# 正規分布の値を持つテンソルを作成
torch.normal(torch.arange(0,100).float())

class AnomalyAttention(nn.Module):
def __init__(self, seq_dim, in_channels, out_channels):
    # ベースクラスを初期化する
    super(AnomalyAttention, self).__init__()

    # 入力チャンネルを出力チャンネルに変換するために、バイアス項を除いた線形レイヤーを作成します。
    self.W = nn.Linear(in_channels, out_channels, bias=False) # 入力チャンネルを出力チャンネルに変換するバイアス項なしの線形層

    # Q, K, V, シグマを与えられた次元のゼロテンソルに初期化する。
    self.Q = self.K = self.V = self.sigma = torch.zeros((seq_dim, out_channels))

    # 出力チャンネルのサイズをd_modelとして設定する
    self.d_model = out_channels

    # シーケンスの長さを n にセット
    self.n = seq_dim

    # P行列とS行列を与えられた次元のゼロテンソルに初期化する
    self.P = torch.zeros((seq_dim, seq_dim))
    self.S = torch.zeros((seq_dim, seq_dim))

def forward(self, x):
    # 入力テンソル x を用いて Q, K, V, シグマ行列を初期化する。
    self.initialize(x)

    # 事前連想行列Pを計算する
    self.P = self.prior_association()

    # 系列結合行列 S を計算する
    self.S = self.series_association()

    # 再構成行列 Z を計算する
    Z = self.reconstruction()

    # 再構成行列を返す
    return Z

def initialize(self, x):
    # 入力テンソル x を線形層 W を用いて変換し、Q, K, V, シグマとして設定する
    self.Q = self.K = self.V = self.sigma = self.W(x)

def prior_association(self):
    # 行と列のインデックスの差の絶対値の行列pを作成する。
    p = torch.from_numpy(
    np.abs(
                    np.indices((self.n,self.n))[0] -
                    np.indices((self.n,self.n))[1]
                    )
    )

    # 行列pとシグマを平均と標準偏差として、ガウス行列を作成する
    gaussian = torch.normal(p.float(), self.sigma[:,0].abs())

    # ガウス行列を、最後の次元に沿った要素の和で割って正規化する
    gaussian /= gaussian.sum(dim=-1).view(-1, 1)

    # 正規化したガウス行列を返す
    return gaussian






- このコードは、異常検知モデルを学習させるためのスクリプトです。
 - モデルの構築と学習に PyTorch ライブラリを使用しているようです。

- コードの最初のブロックは、スクリプト全体で使用されるいくつかのライブラリをインポートしています。
 - その中には、テンソル計算とニューラルネットワークの学習を行うPyTorchのメインライブラリであるtorchや、
 - ニューラルネットワークの構築と学習のためのPyTorchのクラスと関数を提供するtorch.nnとtorch.nn.functionが含まれています。
 - また、設定管理のためのライブラリである hydra と、
 - 複雑な設定データ構造を管理するためのライブラリである omegaconf もインポートしています。

- 2つ目のブロックでは、乱数の配列を作成し、系列の一部に少量のノイズを加えることで合成時系列データを生成しています。
　- この時系列は次にPyTorchテンソルに変換され、time_series変数に格納されます。

- 3番目のブロックでは、100x100グリッドの点間のペアワイズ距離の行列を生成します。
 - この行列はガウシアンカーネルを作成するために使用され、gaussian変数に格納されます。

- 4番目のブロックでは、AnomalyAttentionというクラスを定義しています。
 - これはPyTorchのnn.Moduleクラスのサブクラスです。
 - このクラスは、異常検知モデルのトランスフォーマーブロックを表します。
 - このクラスは、
     - 入力テンソルを新しい空間に投影するWと呼ばれる線形層、
     - 注意のために使われるQ、K、Vと呼ばれるクエリー、キー、値の行列、
     - そしてガウス分布の標準偏差を表すシグマテンソルから構成されています。
     - このクラスはまた、入力テンソルに基づいてQ、K、V、シグマテンソルを初期化するinitializeと、
     - 変換ブロックの前方通過を実行するforwardの2つのメソッドを持っている。

- 5番目のブロックはAnomalyTransformerBlockという別のクラスを定義しており、
 - これもnn.Moduleのサブクラスである。
 - このクラスは、異常検出のためのトランスフォーマーモデルにおける単一のブロックを表します。
 - このクラスには、
     - AnomalyAttention クラスのインスタンスの他に、
     - ニューラルネットワークの活性度を正規化するための ln1 と ln2 という 2 つの層正規化層、線形層と
     -  ReLU 活性化からなる ff というフィードフォワードネットワーク、
     - 事前関連行列と系列関連行列間の KL 発散を計算する association_discrepancy というメソッドが含まれます。

- コードの6番目のブロックはスクリプトのメイン関数を定義しており、異常検出モデルの学習を担当する。
 - この関数は、omegaconfとhydraを使用してモデルの設定を行うことから始まります。
 - 次に、合成時系列データをロードし、PyTorchテンソルに変換します。
 - 次に、複数のAnomalyTransformerBlockブロックを積み重ねた変換モデルをインスタンス化します。
 - 最後に、この関数は、再構成損失と関連性不一致損失を組み合わせた損失関数を用いて、時系列データ上でモデルを学習させる。




- このコードでは、
 - PyTorchでニューラルネットワークを作成・学習するためのtorchと torch.nn、
 - 設定や依存性注入を扱うライブラリであるhydra、
 - 階層的な設定ファイルを扱うライブラリであるomegaconf、
 - データ操作や可視化に使用するsklearnと matplotlibなどのライブラリをいくつかインポートしています。

- 次に、このコードはいくつかのランダムな時系列データを生成し、時系列の部分集合に小さな摂動を適用します。
 - この時系列データはPyTorchテンソルに変換され、time_series変数に格納されます。

- 次に、このコードは(100,100)の形状テンソルの添字間の差の絶対値を表すテンソルpを作成する。
 - 次にテンソルσは長さ100の1の列ベクトルとして作成され、テンソルPと Sは次元(10,10)の1の行列として作成され、それぞれ0から9の値の列と行ベクトルが掛け合わされる。
 - これらのテンソルの目的は明らかではない。

- このコードは、次にtorch.nn.functionalモジュールのkl_div関数をPとS の各行に適用し、そのためのラムダ関数を作ろうとしているよう

- 次にこのコードは、平均p、標準偏差σの正規分布からサンプリングしてテンソルgaussianを作成し、
 - 各要素を最終次元に沿った要素の合計で割ることによってテンソルを正規化する。
 - そして、ガウスの最初の行の和を計算し、それが1に等しいかどうかをチェックする。

- 次に、このコードは次元(5,10)の1のテンソルを作成し、各要素を最終次元に沿った要素の合計で割ることによって正規化する。
 - また、最終次元に沿った要素の和も計算される。

- このコードでは、平均値が0から100の範囲にある正規分布からサンプリングしようとします

- このコードは、AnomalyAttentionというPyTorchのニューラルネットワークモジュールを定義しており、
 - 時系列データに対して何らかの形で異常検知を行うように設計されているようです。
 - このモジュールには、ネットワークの重みとバイアスを初期化する__init__メソッドと、ネットワークのフォワードパスを定義するforwardメソッドがあります。
 - forwardメソッドには、入力データに基づいていくつかのクラス属性の値を設定するinitializeメソッドと、
 - 入力データに基づいてある種の関連行列を作成すると思われるprior_associationメソッドと
 - series_associationメソッドが含まれる。
 - forwardメソッドにはreconstructionメソッドも含まれている

PyTorch ライブラリを使ったコードのようです。いくつかの関数とクラスを定義し、使用しているようです。以下は、このコードで起こっていることの内訳です。

PyTorch ライブラリやそのモジュール、NumPy, matplotlib, sklearn などの他のライブラリなど、いくつかのインポートが行われています。

time_series変数はランダムな値のNumPy配列として初期化され、配列の特定の領域のいくつかの値は固定値に設定されています。この配列はその後、PyTorchテンソルに変換され、time_series変数に格納されます。

変数pは、indices関数から得られる2つのNumPy配列の計算結果を用いて、PyTorchテンソルとして初期化されています。

シグマ変数は、PyTorchのテンソルとして初期化され、(100, 1)の形状を持つように再形成されています。

Pと Sの変数は、(10, 10)の形とtorch.arange関数に依存する値を持つ、1のPyTorchテンソルとして初期化されています。

行番号を入力とし、PテンソルとSテンソルの行で表される確率分布間の2つのKL（Kullback-Leibler）ダイバージェンスの和を返すラムダ関数が定義されつつあります。

Pテンソルをプリントしているところです。

torch.arange関数は、0から10の範囲で、0.1のステップサイズで呼び出されています。

torch.exp関数はシグマテンソルとpテンソルを含む計算の結果に対して呼ばれている。

ガウス変数は、pと シグマを引数としてtorch.normal関数を呼び出した結果から得られた PyTorch テンソルとして初期化されています。このテンソルは、最後の次元に沿った値の総和で割ることによって正規化されます。

ガウステンソルの1行目の和が計算され、プリントされる。

形状が(5, 10)で値がoneの2つのテンソルが作成され、印刷されている。最初のテンソルは最後の次元の値の総和で割ることによって正規化されている。

2番目のテンソルの最後の次元に沿った値の合計が印刷されている。

torch. arange関数で作成したテンソルを引数に、正規分布からの値を持つPyTorchテンソルをtorch.normal関数で作成し、表示しているところです。

PyTorch のnnモジュールのnn.ModuleクラスのサブクラスであるAnomalyAttentionクラスを定義しています。このクラスは、初期化のための__init__メソッド、フォワードパスのためのforwardメソッド、その他いくつかのメソッドを持っています。

prior_associationメソッドが定義されつつあり、indices関数から取得したNumPy配列を含む計算結果を用いて初期化したPyTorchテンソルを返します。